In [4]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# BCCSU Monthly Update

In [4]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from datetime import datetime
import re


today = datetime.today()

start_date = datetime.strptime("2024-06-30", "%Y-%m-%d")

url = "https://bccsu-drugsense.onrender.com/"
max_pages = 100000

driver = webdriver.Chrome()

driver.get(url)

time.sleep(10)

res_button = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/ul/li[3]/a")
ActionChains(driver).move_to_element(res_button).click(res_button).perform()

time.sleep(10)

data = []

page_count = 0
continue_scraping = True

while page_count < max_pages:
    tbody = driver.find_element("xpath", "/html/body/div/div/div[3]/div/div/div[3]/div/div/div[2]/div/div/div/div[2]/div/div[2]/div[2]/table/tbody")
    rows = tbody.find_elements("tag name", "tr")

    for row in rows[2:]:  
        cols = row.find_elements("tag name", "td")

        if cols:
            date_str = cols[0].text
            try:
                row_date = datetime.strptime(date_str, "%Y-%m-%d")
                print(f"Processing date: {row_date}")

                if start_date <= row_date <= today:
                    row_data = [col.text for col in cols]
                    data.append(row_data)
                elif row_date < start_date:
                    print(f"Breaking on date: {row_date}")
                    continue_scraping = False
                    break  
            except ValueError as e:
                print(f"Date conversion error: {e}")

    if not continue_scraping:
        break  

    page_count += 1

    try:
        next_button = driver.find_element("xpath", "/html/body/div[1]/div/div[3]/div/div/div[3]/div/div/div[2]/div/div/div/div[3]/button[3]")
        if "disabled" in next_button.get_attribute("class"):
            break
        else:
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(3)  
    except NoSuchElementException:
        break  
df_bccsu = pd.DataFrame(data, columns=['Date', 'Location', 'Health Authority', 'Drug Checking Site', 'Sold As/Expected Drug', 'Category', 'Colour', 'Texture', 'Fentanyl', 'Benzodiazepine', 'FTIR Spectrometer'])

driver.quit()


def format_color(color):
    match = re.match(r"(\w+)\s*\((dark|light)\)", color, re.IGNORECASE)
    if match:
        return f"{match.group(2).capitalize()} {match.group(1).capitalize()}"
    match_reversed = re.match(r"(dark|light)\s*\((\w+)\)", color, re.IGNORECASE)
    if match_reversed:
        return f"{match_reversed.group(1).capitalize()} {match_reversed.group(2).capitalize()}"
    return color

df_bccsu['Colour'] = df_bccsu['Colour'].apply(format_color)


def transform_result(value):
    if value == 'Pos':
        return 'Positive'
    elif value == 'Neg':
        return 'Negative'
    return value

def specify(value):
    if pd.isna(value) or value == '':
        return 'Unspecified'
    return value


df_bccsu['Fentanyl'] = df_bccsu['Fentanyl'].apply(transform_result)
df_bccsu['Benzodiazepine'] = df_bccsu['Benzodiazepine'].apply(transform_result)
df_bccsu['Province'] = 'BC'
df_bccsu['Fentanyl'] = df_bccsu['Fentanyl'].fillna('Unspecified')
df_bccsu['Benzodiazepine'] = df_bccsu['Benzodiazepine'].fillna('Unspecified')
df_bccsu['Fentanyl'] = df_bccsu['Fentanyl'].apply(specify)
df_bccsu['Benzodiazepine'] = df_bccsu['Benzodiazepine'].apply(specify)
df_bccsu['Source'] = 'bccsu-drugsense.onrender.com'
df_bccsu['Code'] = 'N/A'
df_bccsu['Address'] = df_bccsu['Location'] + ', ' + df_bccsu['Province'] + ', Canada' 
new_column_order = ['Date','Location','Province','Address','Health Authority','Drug Checking Site','Sold As/Expected Drug','Category','Colour','Texture','Code','Fentanyl','Benzodiazepine','FTIR Spectrometer','Source']
df_bccsu = df_bccsu[new_column_order]
df_bccsu = df_bccsu.sort_values(by = 'Date', ascending = False)


Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
Processing date: 2024-06-30 00:00:00
P

# Get Your Drug Tested Monthly Update

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime

today = datetime.today()

start_date = datetime.strptime("06/09/2024", "%m/%d/%Y")

driver = webdriver.Chrome()

base_url = 'https://getyourdrugstested.com/alerts/page/{}'
start_page = 1
max_pages = 100

data = []

page_count = 0

continue_scraping = True

while page_count < max_pages:
    url = base_url.format(start_page + page_count)
    driver.get(url)
    time.sleep(3)  

    rows = driver.find_elements(By.CSS_SELECTOR, '#custom-table tbody tr.alert')
    if not rows:
            print(f"No data found on page {start_page + page_count}. Stopping.")
            break

    for row in rows:
        date = row.find_element(By.CSS_SELECTOR, 'td.date').text.strip()
        location = row.find_element(By.CSS_SELECTOR, 'td.location').text.strip()
        category_description = row.find_element(By.CSS_SELECTOR, 'td.description').text.strip()
        sold_as = row.find_element(By.CSS_SELECTOR, 'td.soldas').text.strip()
        note = row.find_element(By.CSS_SELECTOR, 'td.notes').text.strip()
        fentstrip = row.find_element(By.CSS_SELECTOR, 'td.fentstrip').text.strip()
        result = row.find_element(By.CSS_SELECTOR, 'td.result').text.strip()
        row_data = [date, location, sold_as, category_description, fentstrip, result, note]
        try:
            row_date = datetime.strptime(date, "%m/%d/%Y") 
            print(f"Processing date: {row_date}")
            if start_date <= row_date <= today:
                data.append(row_data)
            elif row_date < start_date:
                print(f"Breaking on date: {row_date}")
                continue_scraping = False
                break  
        except ValueError as e:
            print(f"Date conversion error: {e}")
    if not continue_scraping:
        break  
    page_count += 1




columns = ['Date', 'Location', 'Sold As/Expected Drug','category_description','fentstrip', 'result', 'note']
df_get = pd.DataFrame(data, columns=columns)

driver.quit()

import pandas as pd
import re

def transform_location(location):
    lines = location.split(', ')
    city = lines[0]
    province = lines[1]
    return pd.Series({'Location': city, 'Province': province})

df_get[['Location', 'Province']] = df_get['Location'].apply(transform_location)
def transform_description(description):
    lines = description.split('\n')
    
    category = 'Unspecified'
    color = 'Unspecified'
    texture = 'Unspecified'
    code = 'N/A'
    
    if lines[0].startswith("Category:"):
        category = lines[0].split(": ")[1]
    
    if len(lines) > 1:
        color_texture = lines[1].split()
        if len(color_texture) >= 2 and color_texture[0] in ['Light', 'Dark']:
            color = ' '.join(color_texture[:2])
            texture = ' '.join(color_texture[2:])
        else:
            color = color_texture[0]
            texture = ' '.join(color_texture[1:])
    
    for line in lines[2:]:
        if line.startswith("Code:"):
            code = line.split(": ")[1]
            break

    return pd.Series({'Category': category, 'Color': color, 'Texture': texture, 'Code': code})


df_get[['Category', 'Colour', 'Texture', 'Code']] = df_get['category_description'].apply(transform_description)
df_get.drop(columns=['category_description'], inplace=True)

df_get['Date'] = pd.to_datetime(df_get['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')


def transform_values(fentstrip_value):
    if isinstance(fentstrip_value, str):
        lines = fentstrip_value.split("\n")
        data_dict = {'Fentanyl': 'Unspecified', 'Benzodiazepine': 'Unspecified'}
        for line in lines:
            key, value = line.split(": ")
            if value == 'Positive':
                data_dict[key] = 'Positive'
            elif value == 'Negative':
                data_dict[key] = 'Negative'
            else:
                data_dict[key] = 'Unspecified'
        return pd.Series(data_dict)
    else:
        return pd.Series({'Fentanyl': 'Unspecified', 'Benzodiazepine': 'Unspecified'})

def transform_string(s):
    return re.sub(r'\s*\d+%-\d+%|\s*\d+%|\s*\d+-\d+%', '', s.replace('\n', ', ')).strip(', ')

df_get['result'] = df_get['result'].apply(transform_string)


df_get[['Fentanyl', 'Benzodiazepine']] = df_get['fentstrip'].apply(transform_values)

df_get.drop(columns=['fentstrip'], inplace=True)
df_get['Source'] = 'getyourdrugstested.com'
df_get.drop(columns={'note'}, inplace=True)
df_get.rename(columns={'result': 'FTIR Spectrometer'}, inplace=True)
df_get['Health Authority'] = ''
df_get['Drug Checking Site'] = 'Get Your Drugs Tested'
df_get['Address'] = df_get['Location'] + ', ' + df_get['Province'] + ', Canada' 
new_column_order = ['Date','Location','Province','Address','Health Authority','Drug Checking Site','Sold As/Expected Drug','Category','Colour','Texture','Code','Fentanyl','Benzodiazepine','FTIR Spectrometer','Source']
df_get = df_get[new_column_order]
df_get = df_get.sort_values(by = 'Date', ascending = False)


/Users/seunghyunpark/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Processing date: 2024-07-18 00:00:00
Processing date: 2024-07-13 00:00:00
Processing date: 2024-07-07 00:00:00
Processing date: 2024-06-11 00:00:00
Processing date: 2024-06-11 00:00:00
Processing date: 2024-06-02 00:00:00
Breaking on date: 2024-06-02 00:00:00


# Merge Data

In [10]:
df_monthly = df_bccsu.merge(df_get, how = 'outer')
df_monthly = df_monthly.sort_values(by = 'Date', ascending = False)
df_monthly.to_csv('Monthly_Update.csv', index=False)